# Objective

__Purpose__:
- check dell outlet stock website
- filter for 15inch screen, i7 7..H. Processor, 3840x2160 screen, (scratch and dent?), outlet price < 1200
- send result to personal email

Dell outlet URL<br>

Active filters:
- XPS 15 9560
- i7
- 15-16inch
- 3840x2160

In [2]:
outletURLPage1 = "http://outlet.euro.dell.com/Online/InventorySearch.aspx?c=uk&cs=ukdfh1&l=en&s=dfh&brandid=7&sign=PXhcOSHtr1T4IOw%2fPR7UdQQtu8ahqH%2bNHH7w97RBh24Yb9ytkYEwHZxQv9U80NfLyDpcxEbPA%2bQ4gJotV48WzamWydAzuCBajsdD24yQdkrRWEd0Afm6GvzpUjsdxJe3QVbHjM%2fDzGzVL5PmDUJ26kUrHxrxG3PaSE97ZZ1TESjW%2fExCZjGJtWMImzIJrfMgNSuVc8hA0%2b%2fw3ZVBonDQsW6XaFlvfbawyudtMeLZ6g8MZVlfcCzO%2bND3DYnPDPILCHs12ssROc%2fPc%2fFW24ZEamIN1MGhMTYJ6NXA1WEem161c0A6aF98wKHdZxTp3kbE46wKm%2bthFJnCMcpbz5%2btWYu7up1Ky6JGB4%2bIlqaJ%2bVUfh6UK5i9aPXMIHNXULO7xCYvu69R7sdQ%3d"
outletURLPage2 = "http://outlet.euro.dell.com/Online/InventorySearch.aspx?c=uk&cs=ukdfh1&l=en&s=dfh&brandid=7&sign=PXhcOSHtr1T4IOw%2fPR7UdUbIb1XeAVkg7mqbVtWGF38z5d%2frJiiyf6HqilpYOtgwhuGdJHhtznNyEPM%2bNJySPYY%2bdIYXEh8VwZMPo2V%2bTwZZTv6FkWzNiUiC9qe9eWdtHTNW7E7GxowZnLxEzF2LyabINL9vicwkY9uENvNNsp1X4NuXUtHmKM%2bjQ5Jh3Ya7ENnWnbqbf8wwrf1kE5FD4D8iff70kzAkWbVq72DAntplNqL4A9auTP3GV%2bTR6QZRbEG6%2bz%2bryyovBb2YVzKlVzmhkxaxFd64%2bPFNhXnAb%2fgMXH1O6LsCgWF56qvfA67UgZKsXi2JnRmz8Gzz34i6HwF5b4OOsDmsxVVz4StwovV16QDmkEeK1t9btUtq1aiq"

__References__:
- Coursera: Python for Everyone Series: Using Python to Access Web Data
    - https://www.coursera.org/learn/python-network-data/home/welcome
    - https://www.py4e.com/html3/12-network
    


create output file to hold results

In [3]:
from datetime import datetime

run_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
output_file = "C:\\Users\\charl\\Google Drive\\10. Coding\\201707 Dell Outlet Stock Check with Python\\results\\" + str(run_time) + ".txt"
# a+ will append after creating file (if doesn't exist)
output = open(output_file, "a+")

### get html string back from dell outlet website

reading html using decode then print one line at a time (more human readable)

In [4]:
from urllib.request import urlopen
from datetime import datetime

# html is a HTTPResponse object
html1 = urlopen(outletURLPage1)
html2 = urlopen(outletURLPage2)

# creat a list of offered products
results_string = []

# convert html object into string using decode() then print each line
for line in html1:
    #print(line.decode().strip())
    results_string.append(line.decode().strip())

for line in html2:
    #print(line.decode().strip())
    results_string.append(line.decode().strip()) 
    


ImportError: No module named request

### Parsing html string using regex

In [5]:
from re import search

# list to store all the offers, each offer's details are stored as a dictionary within the master list
offers = []
# index counter for the offers list
j = -1
# offer class
class Offer():
    def __init__(self, condition):
        self.condition = condition


for i in range(len(results_string)):
    if search(r'<div class="fl-item-condition">.*?</div>',results_string[i]):
        condition = search(r'<div class="fl-item-condition">(.*?)</div>',results_string[i]).group(1)
        if condition == "":
            condition = "Certified Refurbished"
        # create a new offer object to capture details of the new offer
        j += 1
        offers.append(Offer(condition))
        
    if search(r'<lft>Outlet Price</lft><rgt>.+?</rgt>',results_string[i]):
        price = search(r'<lft>Outlet Price</lft><rgt>(.+?)</rgt>',results_string[i]).group(1)
        setattr(offers[j], "price", price)
        
    if search(r'<lft>Outlet Price From</lft><rgt>.+?</rgt>',results_string[i]):
        price = search(r'<lft>Outlet Price From</lft><rgt>(.+?)</rgt>',results_string[i]).group(1)
        setattr(offers[j], "price", price)    
        
    if search(r'<div class="fl-config-desc-container">',results_string[i]):
        # by observation, model name is 2 lines below the search line above
        model = search(r'<div>(.+?)</div>',results_string[i+2]).group(1)
        setattr(offers[j], "model", model)
        
        # by observation, model name is 2 lines below the search line above
        processor = search(r'<div>.+?i7-(.+?)</div>',results_string[i+4]).group(1)
        setattr(offers[j], "processor", processor)
        
        # by observation, model name is 2 lines below the search line above
        memory = search(r'<div>(.+?)</div>',results_string[i+8]).group(1)
        setattr(offers[j], "memory", memory)
        
    if search(r'<div>.+?B.+?SSD</div>',results_string[i]):
        SSD = search(r'<div>(.+?B.+?)SSD</div>',results_string[i]).group(1)
        setattr(offers[j], "SSD1", SSD)
        
    if search(r'<div>.+?B.+?Solid State.+?</div>',results_string[i]):
        SSD = search(r'<div>(.+?B).+?Solid State.+?</div>',results_string[i]).group(1)
        setattr(offers[j], "SSD2", SSD)
        
    if search(r'<div>(.+?B).+?Hard Drive.+?</div>',results_string[i]):
        HD = search(r'<div>(.+?B).+?Hard Drive.+?</div>',results_string[i]).group(1)
        setattr(offers[j], "HD", HD)
        
    if search(r'<div>.+?cell(.+?)Battery</div>',results_string[i]):
        battery = search(r'<div>.+?cell\s(.+?) Battery</div>',results_string[i]).group(1)
        setattr(offers[j], "battery", battery)





    

## Output Results

In [6]:
output.write("run time: " + str(run_time))
output.write("\n")

num_offers = len(offers)
output.write("number of offers found: "+str(num_offers))
output.write("\n")

# second offers list only containing 7xxx procesor
offers_7 = []
for i in range(num_offers):
    if search(r'^7',offers[i].processor):
        offers_7.append(offers[i])
        
num_offers_7 = len(offers_7)
output.write("number of 7xxx processor offers found: "+str(num_offers_7))
output.write("\n")

output.write("\n")
output.write("======================== offers with 7xxx processor: ===========================")  
output.write("\n")

for i in range(num_offers_7):
    output.write(offers_7[i].model)
    output.write("\n")
    output.write(offers_7[i].processor)
    output.write("\n")
    
    storage = ""
    if hasattr(offers_7[i], "SSD1"):
        storage = storage + offers_7[i].SSD1 + "SSD"
    if hasattr(offers_7[i], "SSD2"):
        storage = storage + offers_7[i].SSD2 + "SSD"
    if hasattr(offers_7[i], "HD"):
        storage = storage + offers_7[i].HD + "HD"
    storage.strip()
    output.write(storage)
    output.write("\n")
   
    output.write(offers_7[i].price)
    output.write("\n")
    output.write(offers_7[i].condition)
    output.write("\n")
    output.write(offers_7[i].memory)
    output.write("\n")
    output.write("\n")
    

output.write("================================== all offers ====================================")
output.write("\n")

for i in range(num_offers):
    output.write(offers[i].model)
    output.write("\n")
    output.write(offers[i].processor)
    output.write("\n")
    
    storage = ""
    if hasattr(offers[i], "SSD1"):
        storage = storage + offers[i].SSD1 + "SSD"
    if hasattr(offers[i], "SSD2"):
        storage = storage + offers[i].SSD2 + "SSD"
    if hasattr(offers[i], "HD"):
        storage = storage + offers[i].HD + "HD"
    storage.strip()
    output.write(storage)
    output.write("\n")
   
    output.write(offers[i].price)
    output.write("\n")
    output.write(offers[i].condition)
    output.write("\n")
    output.write(offers[i].memory)
    output.write("\n")
    output.write("\n")

output.close()

# Send results to Email

In [ ]:
import win32com.client
olMailItem = 0x0
obj = win32com.client.Dispatch("Outlook.Application")
newMail = obj.CreateItem(olMailItem)
newMail.Subject = "My Subject"
newMail.Body = "My Body"
newMail.To  = "charleszhao@hotmail.co.uk"
newMail.Send()

### Target html section containing key information

typical per product string returned (naturally readable in jupyter as it's writen in xml)

<div class=" ">
<ul>
<li>
<a href="../Online/SecondaryInventorySearch.aspx?c=uk&cs=ukdfh1&l=en&s=dfh&sign=PXhcOSHtr1T4IOw%2fPR7UdfocNBLlHp1qPdu4uUw4j%2bDERNVxM9c6fzEuhK63ccjlrxD%2f65uY%2bsMmxx1oBLb%2fOmJu2g7W%2b4GPVrOzhlw4eVCOaJiBXlvlCGNci7ThcENUBpZzSen36vd5nRd0weV7%2fZF0zclRa9lfZw1%2fgFt%2bKPCHdFBab32e7wm2YB6AvXYfdNMuPKK0ZLHBRcizU7AB14ygwmRp3aERO0Rx7qV2KaL6HGXuv6J9QA%3d%3d">
<img class="image-centered widthMax216 fl-config-img-head" src='http://i.dell.com/sites/imagecontent/consumer/merchandizing/en/PublishingImages/Outlet_Stockroom_Images/XPS_15_9550.jpeg' alt='XPS 15 - 9550' title='XPS 15 - 9550'>
</a>
</li>
<li>
<h4 class="fl-sub-head">
<a class="fl-config-caption" href="../Online/SecondaryInventorySearch.aspx?c=uk&cs=ukdfh1&l=en&s=dfh&sign=PXhcOSHtr1T4IOw%2fPR7UdfocNBLlHp1qPdu4uUw4j%2bDERNVxM9c6fzEuhK63ccjlrxD%2f65uY%2bsMmxx1oBLb%2fOmJu2g7W%2b4GPVrOzhlw4eVCOaJiBXlvlCGNci7ThcENUBpZzSen36vd5nRd0weV7%2fZF0zclRa9lfZw1%2fgFt%2bKPCHdFBab32e7wm2YB6AvXYfdNMuPKK0ZLHBRcizU7AB14ygwmRp3aERO0Rx7qV2KaL6HGXuv6J9QA%3d%3d">
XPS 15 - 9550<br/>£1,029.26
</a>
</h4>
<span>1 in stock</span>
</li>
<li>
<div class="fl-item-condition">Scratch and Dent</div>
<div class="fl-config-desc-container">

<div>XPS 15 - 9550</div>

<div>Processor: Intel&#174; Core™ i7-6700HQ (4 Core,6M Cache, up to 3.5GHz, 45W)</div>

<div>Windows 10 Home (64bit)</div>

<div>16GB (2x8GB) 2133MHz DDR4 Non-ECC</div>

<div>512GB PCIe Solid State Drive</div>

<div>2 GB GDDR5 Nvidia&#174; GeForce&#174; GTX 960M</div>

<div>Software</div>

<div>Dell Wireless-1830 802.11ac, 3x3,2.4/5GHz + Bluetooth 4.1</div>

<div>Power Cord : 250V</div>

<div>Primary 6-cell 84W/HR Battery</div>

<div>15.6inch 4K Ultra HD (3840 x 2160) &#160;InfinityEdge Touch Display</div>

<div>Internal Qwerty Backlit Keyboard</div>

</div>
</li>
<li>
<div class="fl-line-aaa"></div>
</li>
<li>
<lft>Outlet Price</lft><rgt>£1,029.26</rgt>
</li>
<li>
<div class="fl-line-aaa"></div>
</li>
<li>
<a href="Handler/SecondaryLikeConfigHandler.ashx?c=uk&cs=ukdfh1&l=en&s=dfh&sign=CrZnLRo%2faTdtpio9U2lfpWlIEPp77pPLdKpMMr50ol64LjcegAITQK0CejEOj9OGlT34dd7jcBN%2bMTEc73m3C2IVW%2fPNtvMc%2b3kLl%2fPLfWXitALDjlp%2bgBvxJDNAjGIXJXbaW8zOv%2fhRMd8HhpsgXTq4J%2bs8uavTbMlpF%2bAnFf8igKc5VbYACI%2fAhUrbPjvcIhbbl2o5xHfebVn%2b2bIVqrPEzwzbF0zDQRyd3HgQpqFPCrw8cTzIVN%2biY6Jk7fWJ" class="fl-link-button">Add to Cart</a>
</li>
</ul>
</div>

# Send cmd from python script

In [ ]:
import subprocess

script_folder = "C:\\Users\\charl\\Google Drive\\10. Coding\\201707 Dell Outlet Stock Check with Python\\"
subprocess.call(['py','-3',script_folder + 'Dell_Outlet_Stock_Checker_v1.py'])
subprocess.call(['py','-2',script_folder + 'Send_Email_v1.py'])

# Reading from results

open(file,mode) is mode is "rb" then will read in bytes without decoding, use "r" for read only

In [8]:
content = open(r'C:\Users\charl\Google Drive\10. Coding\201707 Dell Outlet Stock Check with Python\results\2017-07-23-19-13-55.txt', 'r').read()

type(content)

print(content)

run time: 2017-07-23-19-13-55
number of offers found: 11
number of 7xxx processor offers found: 5

======================== offers with 7xxx processor: ===========================
XPS 15 - 9560
7700HQ (Quad Core,up to 3.8GHz, 6MB Cache, 45W)
512GBSSD
£1,259.32
Certified Refurbished
16GB DDR4 SDRAM (2x8GB) 2400MHz Non-ECC

XPS 15 - 9560
7700HQ (Quad Core,up to 3.8GHz, 6MB Cache, 45W)
512GBSSD
£1,265.32
Certified Refurbished
16GB DDR4 SDRAM (2x8GB) 2400MHz Non-ECC

XPS 15 - 9560
7700HQ (Quad Core,up to 3.8GHz, 6MB Cache, 45W)
512GBSSD
£1,285.33
Certified Refurbished
16GB DDR4 SDRAM (2x8GB) 2400MHz Non-ECC

XPS 15 - 9560
7700HQ (Quad Core,up to 3.8GHz, 6MB Cache, 45W)
1TBSSD
£1,489.38
Certified Refurbished
32GB (2x16GB) DDR4 SDRAM 2400MHz Non-ECC

XPS 15 - 9560
7700HQ (Quad Core,up to 3.8GHz, 6MB Cache, 45W)
1TBSSD
£1,515.39
Certified Refurbished
32GB (2x16GB) DDR4 SDRAM 2400MHz Non-ECC

================================== all offers ====================================
XPS 15 - 9550
6700H

# Scheduling work at Interval

In [ ]:
import subprocess
import time
from datetime import datetime

def run(run_interval):
    start_time = time.time()
    run_time = datetime.now()

    interval_min = run_interval

    output_file = scan()
    email_results(output_file)

    while True:
      print("scan completed at " + str(run_time))
      print("next scan scheduled in " + str(interval_min) + " mins")
      print
      time.sleep(60.0 * interval_min - (time.time() - start_time))

run(1)

simple test

In [4]:
import time

start_time = time.time()
time.sleep(5)

time_taken = time.time()-start_time
target_interval = 60.0 * 1
time_until_next = target_interval - time_taken

#time.sleep(target_interval - time_taken)

print(time_until_next)

time.sleep(time_until_next)

print("time is up")

54.99178910255432
time is up
